# Submission

In [2]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [3]:
submission_with_whole_train = False

# Feature engineer (option 1)

In [ ]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_extra = pd.read_csv("data/stores_extra.csv")
if not submission_with_whole_train:
    stores_train, stores_val = train_test_split(stores_train, test_size=0.2, random_state=0)

In [ ]:
from utils import split_plaace_cat


store_dataframes = {
    "train": stores_train, 
    "extra": stores_extra, 
    "test": stores_test, 
    }

if not submission_with_whole_train:
    store_dataframes["val"] = stores_val

for df_name, df in store_dataframes.items():
    df = split_plaace_cat(df)

In [ ]:
store_dataframes["train"].shape

In [ ]:
store_dataframes["train"]["log_revenue"] = store_dataframes["train"].revenue.apply(lambda x: np.log1p(x))
if not submission_with_whole_train:
    store_dataframes["val"]["log_revenue"] = store_dataframes["val"].revenue.apply(lambda x: np.log1p(x))

In [ ]:
from utils import create_geographical_columns, create_chain_and_mall_columns, generate_rev_dict, generate_chain_rev_dict, create_mean_chain_rev_col

chain_count = stores_train["chain_name"].value_counts().to_dict()
lower_limit = 1
rev_dict, mean_revenue = generate_rev_dict(store_dataframes["train"])

for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_geographical_columns(df)
    store_dataframes[df_name] = create_chain_and_mall_columns(df, chain_count, lower_limit=lower_limit)

chain_rev_dict, log_bounded_chain_rev_dict = generate_chain_rev_dict(store_dataframes["train"])
for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_mean_chain_rev_col(df, bounded_chain_revs=chain_rev_dict, log_bounded_chain_revs=log_bounded_chain_rev_dict)


In [ ]:
from utils import concat_df_keep_unq_index
concat_df = concat_df_keep_unq_index(stores_train, stores_extra)

!NB next cell takes 3 minutes to run (if comp_plaace_gran. = [1, 2, 3, 4]) (on M1 Mac with 16GB RAM)

(outdated) If you have the .csv files temp_data/closest_comp_\{df_name\}, skip running this cell and run the cell below it instead

In [ ]:
from utils import find_dist_to_nearest_comp

nearest_comp_plaace_cat_gran = [1, 2, 3, 4]
n_nearest_comp = [1, 2, 3, 4, 5, 7, 10]

store_dataframes["train"] = find_dist_to_nearest_comp(
    store_dataframes["train"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=True, 
    training_df=concat_df,
    )

In [ ]:
if not submission_with_whole_train:
    store_dataframes["val"] = find_dist_to_nearest_comp(
        store_dataframes["val"], 
        nearest_comp_plaace_cat_gran, 
        n_nearest_comp, 
        training=True, 
        training_df=concat_df,
        )

In [ ]:
store_dataframes["test"] = find_dist_to_nearest_comp(
    store_dataframes["test"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=False,
    training_df=concat_df
)

In [ ]:
comp_plaace_cols = list(store_dataframes["train"].columns[-56:])

In [ ]:
dist_dict = store_dataframes["train"][['log_revenue'] + comp_plaace_cols].corr().iloc[0].to_dict()
sorted_relevant_dist_cols = [[k, v] for k, v in sorted(dist_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
sorted_relevant_dist_cols

In [ ]:
comp_relevant_cols = sorted_relevant_dist_cols[1:14:2]
comp_relevant_cols = [r[0] for r in comp_relevant_cols]

In [ ]:
comp_relevant_cols

!NB next cell takes 15 minutes to run. Grab a coffee or somtething while you wait :) (on M1 Mac with 16GB RAM)

If you have the .csv files temp_data/closest_bus_stop_\{df_name\}, skip running this cell and run the cell below it instead

In [ ]:
from bus_utils import find_closest_bus_stop

bus_stop_n = [1, 3, 5, 7]
bus_mean = True
bus_sum = True

bus_stop_columns = []

if(bus_sum):
    bus_stop_columns += [f"closest_bus_stop_sum_{i}" for i in bus_stop_n]

if(bus_mean):
    bus_stop_columns += [f"closest_bus_stop_mean_{i}" for i in bus_stop_n]


for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = find_closest_bus_stop(df, bus_stop_n, _sum=bus_sum, _mean=bus_mean)    

In [ ]:
bus_dict = store_dataframes["train"][['log_revenue'] + bus_stop_columns].corr().iloc[0].to_dict()
bus_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(bus_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
bus_sorted_relevant_dist_cols

In [ ]:
bus_relevant_cols = bus_sorted_relevant_dist_cols[1:9:2]
bus_relevant_cols = [r[0] for r in bus_relevant_cols]

In [ ]:
bus_relevant_cols

In [ ]:
from grunnkrets import make_grunnkrets_df

full_population_dataframes = {}
full_pop_columns = []

for df_name, df in tqdm(store_dataframes.items()):
    full_population_dataframes[df_name] = make_grunnkrets_df(df)
    full_pop_columns = full_population_dataframes[df_name].columns

In [ ]:
full_pop_columns = list(full_pop_columns[-184:])

In [ ]:
for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = df.merge(
        full_population_dataframes[df_name], 
        left_index=True,
        right_index=True,
        how="outer", 
        suffixes=('', '_redundant')
    )
    store_dataframes[df_name].drop(store_dataframes[df_name].filter(regex='_redundant$').columns, axis=1, inplace=True)

In [ ]:
full_pop_dict = store_dataframes["train"][['log_revenue'] + full_pop_columns].corr().iloc[0].to_dict()
full_pop_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(full_pop_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
full_pop_sorted_relevant_dist_cols

In [ ]:
full_pop_relevant_cols = full_pop_sorted_relevant_dist_cols[1:8]
full_pop_relevant_cols = [r[0] for r in full_pop_relevant_cols]

In [ ]:
full_pop_relevant_cols

In [ ]:
fylke_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("fylke.")]
kommune_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("kommune.")]
delomrade_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("delomrade.")]
grunnkrets_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("grunnkrets_id.")]

In [ ]:
from utils import mean_func_rev, generate_rev_dict

for i in range(1, 5):
    rev_dict, mean_revenue = generate_rev_dict(stores_train, i)
    for df_name, df in store_dataframes.items():
        df["mean_revenue_" + str(i)] = df["plaace_cat_" + str(i)].apply(lambda x: mean_func_rev(x, rev_dict, mean_revenue))
        df["log_mean_revenue_" + str(i)] = df["mean_revenue_" + str(i)].apply(lambda x: np.log1p(x))

In [ ]:
from num_stores import add_num_stores_info

for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_num_stores_info(df)

In [ ]:
num_store_cols = list(store_dataframes["train"].columns[-64:])

In [ ]:
num_store_dict = store_dataframes["train"][['log_revenue'] + num_store_cols].corr().iloc[0].to_dict()
num_store_sorted_relevant_cols = [[k, v] for k, v in sorted(num_store_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
num_store_sorted_relevant_cols

In [ ]:
num_store_relevant_cols = num_store_sorted_relevant_cols[1:15]
num_store_relevant_cols = [r[0] for r in num_store_relevant_cols]

## Adding mean revenue (plaace_cat, level)

In [ ]:
from avg_revenue import add_avg_revenue

for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_avg_revenue(df, total=submission_with_whole_train)

In [17]:
mean_rev_cols = list(store_dataframes["train"].columns)[-20:]

In [19]:
mean_rev_dict = store_dataframes["val"][['log_revenue'] + mean_rev_cols].corr().iloc[0].to_dict()
mean_rev_sorted_relevant_cols = [[k, v] for k, v in sorted(mean_rev_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
mean_rev_sorted_relevant_cols

[['log_revenue', 1.0],
 ['country.plaace_cat_4_mean_revenue_log', 0.5504002855510065],
 ['fylke.plaace_cat_4_mean_revenue_log', 0.5196944562059945],
 ['country.plaace_cat_3_mean_revenue_log', 0.5171030426373732],
 ['fylke.plaace_cat_3_mean_revenue_log', 0.4875189785102404],
 ['kommune.plaace_cat_4_mean_revenue_log', 0.463005256668693],
 ['delomrade.plaace_cat_4_mean_revenue_log', 0.4425709147424487],
 ['grunnkrets_id.plaace_cat_4_mean_revenue_log', 0.4364211801239641],
 ['kommune.plaace_cat_3_mean_revenue_log', 0.42144640988132864],
 ['country.plaace_cat_2_mean_revenue_log', 0.4133006273221649],
 ['fylke.plaace_cat_2_mean_revenue_log', 0.40369387566866716],
 ['delomrade.plaace_cat_3_mean_revenue_log', 0.38436257812286523],
 ['grunnkrets_id.plaace_cat_3_mean_revenue_log', 0.3772813654612513],
 ['kommune.plaace_cat_2_mean_revenue_log', 0.3263086121465885],
 ['country.plaace_cat_1_mean_revenue_log', 0.3214119161325694],
 ['fylke.plaace_cat_1_mean_revenue_log', 0.3196151731960934],
 ['delo

In [20]:
mean_rev_relevant_cols = mean_rev_sorted_relevant_cols[1:]
mean_rev_relevant_cols = [r[0] for r in mean_rev_relevant_cols]

In [21]:
mean_rev_relevant_log_cols = []
for col in mean_rev_relevant_cols:
    for df_name, df in store_dataframes.items():
        store_dataframes[df_name][col + "_log"] = store_dataframes[df_name][col].apply(lambda x: np.log1p(x))
    mean_rev_relevant_log_cols.append(col + "_log")

# Removing outliers

In [64]:
from outlier_utils import remove_low_revenue

store_dataframes["train"] = remove_low_revenue(store_dataframes["train"], low_rev_limit=0.05)

# Read data directly from CSV (option 2)

## Train, val split

In [61]:
if submission_with_whole_train:
    stores_train = pd.read_csv("temp_data/full_train_features_train.csv", index_col=0)
    stores_extra = pd.read_csv("temp_data/full_train_features_extra.csv", index_col=0)
    stores_test = pd.read_csv("temp_data/full_train_features_test.csv", index_col=0)

    store_dataframes = {
        "train": stores_train, 
        "extra": stores_extra, 
        "test": stores_test, 
        }
else:
    stores_train = pd.read_csv("temp_data/full_features_train.csv", index_col=0)
    stores_val = pd.read_csv("temp_data/full_features_val.csv", index_col=0)
    stores_extra = pd.read_csv("temp_data/full_features_extra.csv", index_col=0)
    stores_test = pd.read_csv("temp_data/full_features_test.csv", index_col=0)

    store_dataframes = {
        "train": stores_train, 
        "extra": stores_extra, 
        "test": stores_test, 
        "val": stores_val
        }

In [5]:
with open("relevant_cols.txt", "r") as f:
    data = f.readlines()

comp_relevant_cols = data[0].strip().split(",")
bus_relevant_cols = data[1].strip().split(",")
num_stores_relevant_cols = data[2].strip().split(",")
full_pop_relevant_cols = data[3].strip().split(",")

In [6]:
comp_relevant_cols = comp_relevant_cols[::2]

In [7]:
comp_relevant_cols += [
    'sum_dist_to_nearest_10_comp_plaace_2',
    'sum_dist_to_nearest_5_comp_plaace_2', 
    'sum_dist_to_nearest_10_comp_plaace_1',
    'sum_dist_to_nearest_5_comp_plaace_1', 
    'sum_dist_to_nearest_1_comp_plaace_1',
    'sum_dist_to_nearest_1_comp_plaace_2'
    ]

In [8]:
bus_relevant_cols = bus_relevant_cols[::1]

In [9]:
num_stores_relevant_cols

['fylke.plaace_cat_1_count',
 'fylke.plaace_cat_4_count',
 'fylke.plaace_cat_3_count',
 'delomrade.plaace_cat_1_count',
 'kommune.plaace_cat_1_count',
 'grunnkrets_id.plaace_cat_1_count']

# Transforming the data

In [10]:
from RMSLE import rmsle
from pred_var_utils import reverse_log1p_transform_pred_var
from sklearn.model_selection import GridSearchCV

In [68]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


OE_categorical_features = ["bounded_chain_name", "kommune", "delomrade", "is_grocery", "plaace_cat_2", "plaace_cat_3", "plaace_cat_4"]
OE_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

OH_categorical_features = ["plaace_cat_1"] #["fylke", "plaace_cat_2"]
OH_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)


numerical_features = ["lat", "lon", 
#"mean_revenue_1", "mean_revenue_2", "mean_revenue_3", "mean_revenue_4", 
"log_mean_revenue_1", "log_mean_revenue_2", "log_mean_revenue_3", "log_mean_revenue_4", 
"fylke.plaace_cat_1_mean_revenue_log", #"fylke.plaace_cat_3_mean_revenue_log",
"log_chain_mean_revenue"
] + full_pop_relevant_cols + comp_relevant_cols + bus_relevant_cols + num_stores_relevant_cols #+ mean_rev_relevant_log_cols
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")), 
        ("scaler", StandardScaler(with_mean=True, with_std=True))]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)

preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)


X_train = preprocessor.fit_transform(store_dataframes["train"])
if not submission_with_whole_train:
    X_val = preprocessor.transform(store_dataframes["val"])

In [69]:
y_train = np.array(store_dataframes["train"].log_revenue)
if not submission_with_whole_train:
    y_val = np.array(store_dataframes["val"].revenue)
mean_y = y_train.mean()
std_y = y_train.std()

y_train -= mean_y
y_train /= std_y

# PCA

In [ ]:
PCA_cols_dict = {
    "OE": [],
    "OH": [],  
    "num": []
}

reduntant_cols = ["store_id", "year", "store_name", "plaace_hierarchy_id", "sales_channel_name", "address", "revenue", "log_revenue", "point"]

for col_name, dtype in store_dataframes["train"].dtypes.to_dict().items():
    if(col_name in reduntant_cols):
        continue
    if(dtype == int or dtype == float):
        _type = "num"
    elif(dtype == bool):
        _type = "OE"
    elif(dtype == object):
        if(store_dataframes["train"][col_name].nunique() <= 10):
            _type = "OH"
        else:
            _type = "OE"
    else:
        print(f"Unknown type {dtype} encountered for columns {col_name}")
    PCA_cols_dict[_type].append(col_name)
    

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, PCA_cols_dict["OE"]),
        ("oh_cat", OH_categorical_transformer, PCA_cols_dict["OH"]),
        ("num", numerical_transformer, PCA_cols_dict["num"]),
    ],
    remainder='drop'
)

In [ ]:
pre_PCA_X_train = preprocessor.fit_transform(store_dataframes["train"])
if not submission_with_whole_train:
    pre_PCA_X_val = preprocessor.transform(store_dataframes["val"])

In [ ]:
pca = PCA(n_components=25)
PCA_X_train = pca.fit_transform(pre_PCA_X_train)
if not submission_with_whole_train:
    PCA_X_val = pca.transform(pre_PCA_X_val)

In [ ]:
print(pca.explained_variance_ratio_)

print(pca.singular_values_)

# Training the model

## Random Forest

In [70]:
from sklearn.ensemble import RandomForestRegressor

# best params (8.11)
# {'max_features': 8, 'min_samples_leaf': 4, 'min_samples_split': 32, 'n_estimators': 500}
# rmsle(on val): 0.719099511243053

rf = RandomForestRegressor(random_state=0, n_jobs=-1, max_features=8, min_samples_leaf=4, min_samples_split=32, n_estimators=500)
rf_params = {
    "n_estimators" : (100, 250, 500), 
    "max_features" : (2, 4, 8), 
    "min_samples_split" : (4, 8, 16, 32), 
    "min_samples_leaf" : (1, 2, 4), 
    }

rf_clf = GridSearchCV(rf, rf_params, verbose=2)

In [71]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_features=8, min_samples_leaf=4, min_samples_split=32,
                      n_estimators=500, n_jobs=-1, random_state=0)

In [72]:
#rf_clf.best_params_

In [73]:
#rf_clf.best_params_

In [74]:
rf_y_pred = reverse_log1p_transform_pred_var(rf.predict(X_val), std_y=std_y, mean_y=mean_y)


In [75]:
rmsle(y_val, rf_y_pred)

0.7199738385597589

## Linear Regression

In [76]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(n_jobs=-1)

In [77]:
lr.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [78]:
lr_y_pred = reverse_log1p_transform_pred_var(lr.predict(X_val), std_y, mean_y)
lr_y_pred = np.array([max(0, xi) for xi in lr_y_pred])

In [79]:
rmsle(y_val, lr_y_pred)

0.7398128593493832

## Light GBM

In [80]:
from lightgbm import LGBMRegressor

# best params (8.11)
# {'learning_rate': 0.05, 'min_child_samples': 16, 'min_split_gain': 0, 'n_estimators': 100, 'num_leaves': 25}
# rmsle(on val): 0.7208999886795342

lgbm = LGBMRegressor(
    random_state=0, 
    n_jobs=-1, 
    learning_rate=0.05, 
    min_child_samples=16, 
    min_split_gain=0, 
    n_estimators=100, 
    num_leaves=25
    )

lgbm_params = {
    "num_leaves" : (10, 25, 31, 75), 
    "learning_rate" : (0.05, 0.1, 0.25), 
    "n_estimators" : (50, 100, 250), 
    "min_split_gain" : (0, 0.01), 
    "min_child_samples" : (4, 8, 16, 32), 
    #"reg_alpha" : (0, 0.01, 0.1), 
    #"reg_lambda" : (0, 0.01, 0.1), 
    }

lgbm_clf = GridSearchCV(lgbm, lgbm_params, verbose=2)

In [81]:
lgbm.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.05, min_child_samples=16, min_split_gain=0,
              num_leaves=25, random_state=0)

In [82]:
#lgbm_clf.best_params_

In [83]:
lgbm_y_pred = reverse_log1p_transform_pred_var(lgbm.predict(X_val), std_y, mean_y)
lgbm_y_pred = np.array([max(0, xi) for xi in lgbm_y_pred])

In [84]:
rmsle(y_val, lgbm_y_pred)

0.7216066193226834

## CatBoost

In [85]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV


# best params (8.11)
# {'depth': 6, 'l2_leaf_reg': 5, 'learning_rate': 0.05, 'eval_metric': 'RMSE'}
# rmsle(on val) = 0.7190096805219454

cb = CatBoostRegressor(
    random_seed=0, 
    verbose=False, 
    eval_metric="RMSE", 
    depth=6, 
    l2_leaf_reg= 5, 
    learning_rate= 0.05
    )

cb_params = grid = {
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.3],
    'depth': [5, 6, 8, 10],
    'l2_leaf_reg': [4, 5, 6, 7, 8], 
    }

# cb_clf = cb.grid_search(cb_params, X=X_train, y=y_train)

In [86]:
#cb_clf["params"]

In [87]:
cb.fit(X_train, y_train)

In [88]:
cb_y_pred = reverse_log1p_transform_pred_var(cb.predict(X_val), std_y, mean_y)
cb_y_pred = np.array([max(0, xi) for xi in cb_y_pred])

In [89]:
rmsle(y_val, cb_y_pred)

0.7202922594712301

## Stacking classifiers

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', rf), 
    ('cb', cb), 
    ('lgbm', lgbm), 
]

rf_end_params = {
    "n_estimators" : (50, 100, 250), 
    "max_features" : (1, 2, 3), 
    "min_samples_split" : (16, 32), 
    "min_samples_leaf" : (2, 4, 8), 
    }

rf_end = RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=50, max_features=3)

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=rf_end
    )

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
reg_y_pred = reverse_log1p_transform_pred_var(reg.predict(X_val), std_y, mean_y)

In [ ]:
rmsle(y_pred=reg_y_pred, y_true=y_val)

## Creating the submission

In [134]:
# Predict on the test set 
X_test = preprocessor.transform(store_dataframes["test"])
y_test_pred = reverse_log1p_transform_pred_var(cb.predict(X_test), std_y, mean_y)

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test.store_id 
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submission.csv', index=False)

# Create CSV of dataframes

In [ ]:
for df_name, df in store_dataframes.items():
    filepath = Path(f"temp_data/full_features_{df_name}.csv")  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath, index=True)